In [1]:
from flask import Flask
from flask_cors import CORS
from flask import request
import json
import pandas as pd
import pathlib
import heapq

from fastai.collab import *
from fastai.tabular.all import *

Microsoft Visual C++ Redistributable is not installed, this may lead to the DLL load failure.
                 It can be downloaded at https://aka.ms/vs/16/release/vc_redist.x64.exe


In [2]:
path_playlists = r"D:\schule\diplomarbeit\converted_json\minimized_mpd_0-9999.json"
path_unique_tracks = r"D:\schule\diplomarbeit\converted_json\unique_tracks_0-9999.json"

path_model = Path("D:/schule/diplomarbeit/models")

In [3]:
save = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath   
learn = load_learner(path_model/'playlists_tracks_br_model_v1.pkl') 
pathlib.PosixPath = save

track_factors = learn.model.i_weight.weight
playlist_factors = learn.model.u_weight.weight

dls = learn.dls

In [4]:
playlists_info = json.load(open(path_playlists, 'r'))
for playlist in playlists_info:
    del playlist['tracks']
playlists = json.load(open(path_playlists, 'r'))
unique_tracks = json.load(open(path_unique_tracks, 'r'))

In [5]:
app = Flask("flowsy")
CORS(app)

@app.route('/search/<filter>')
def search(filter):
    playlist_info = []
    for playlist in playlists_info:
        if(filter in playlist['name']):
            playlist_info.append(playlist)
    return json.dumps(playlist_info)

@app.route('/playlist/<pid>')
def getPlaylistInfo(pid):
    return playlists[int(pid)]

@app.route('/tracks/<pid>')
def getTracksOfPlaylist(pid):
    return json.dumps(playlists[int(pid)]['tracks'])

@app.route('/recommendation/<pid>')
def getRecommendation(pid):
    idx = dls.classes['pid'].o2i[int(pid)]
    distances = np.array(list((playlist_factors[idx] @ track_factor for track_factor in track_factors)))
    n = int(request.args.get('n'))
    temp = n + playlists[int(pid)]['num_tracks']
    indices = heapq.nlargest(temp, range(len(distances)), distances.take)
    tracks_of_playlist = list(track['track_uri'] for track in playlists[int(pid)]['tracks'])
    top_track_uris = list((dls.classes['track_uri'][idx] for idx in indices))
    filtered_top_track_uris = [top_track_uri for top_track_uri in top_track_uris if not top_track_uri in tracks_of_playlist]
    top_tracks = []
    for track_uri in filtered_top_track_uris[:n]:
        for track in unique_tracks:
            if(track['track_uri'] == track_uri):
                top_tracks.append(track)
                break
    return json.dumps(top_tracks)
                
    
app.run(host='0.0.0.0')

 * Serving Flask app "flowsy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
10.0.0.4 - - [16/Nov/2020 17:59:42] "GET /search/pop HTTP/1.1" 200 -
10.0.0.4 - - [16/Nov/2020 17:59:44] "GET /playlist/708 HTTP/1.1" 200 -
10.0.0.4 - - [16/Nov/2020 17:59:44] "GET /tracks/708 HTTP/1.1" 200 -
10.0.0.4 - - [16/Nov/2020 17:59:50] "GET /search/pop HTTP/1.1" 200 -
10.0.0.4 - - [16/Nov/2020 18:04:26] "GET /search/oldies HTTP/1.1" 200 -
10.0.0.4 - - [16/Nov/2020 18:05:06] "GET /search/rap HTTP/1.1" 200 -
10.0.0.4 - - [16/Nov/2020 18:05:08] "GET /playlist/81 HTTP/1.1" 200 -
10.0.0.4 - - [16/Nov/2020 18:05:08] "GET /tracks/81 HTTP/1.1" 200 -
10.0.0.4 - - [16/Nov/2020 18:05:12] "GET /search/rap HTTP/1.1" 200 -
10.0.0.4 - - [16/Nov/2020 18:05:17] "GET /playlist/81 HTTP/1.1" 200 -
10.0.0.4 - - [16/Nov/2020 18:05:17] "GET /tracks/81 HTTP/1.1" 200 -
10.0.0.4 - - [16/Nov/2020 18:06:41] "GET /search/indie HTTP/1.1" 200 -
10.0.0.4 - - [16/Nov/2020 18:06:48] "GET /playlist/3437 HTTP/1.1" 200 -
10.0.0.4 - - [16/Nov/2020 18:06:48]